# Examining the effect of including glacial runoff in SPEI drought risk calculations

Welcome!  This notebook will reproduce and visualise the analyses behind Ultee, Coats & Mackay, "Glacial runoff buffers drought through the 21st century" (submitted to ESD), using helper functions from gSPEI.py.  All code is stored in a public GitHub repository--click [here](https://github.com/ehultee/glacial-SPEI) for access.

### Loading in modules and data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.patches import Rectangle
import gSPEI as gSPEI

In [ ]:
## Define the filepath - abbreviations indicate
## (P)arametric or (NP)nonparametric;
## Standardization (1) lumped or (2) split by starting month
fpath_NP2 = './data/SPEI_Files/nonparametric-var_stom_c/'

## Settings in filenames
integration_times = np.arange(3, 28, 4) # all SPEI integration times used
modelnames = ['CanESM2', 'CCSM4', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'GISS-E2-R', 'INMCM4', 'MIROC-ESM', 'NorESM1-M'] # all models used in comparison
scenarios = ['Rcp4p5', 'Rcp8p5'] # climate scenarios
cases = ['NRunoff', 'WRunoff', 'diff'] # inclusion of glacier runoff

## Basins in the order they are written in each file
basin_names = ['INDUS','TARIM','BRAHMAPUTRA','ARAL SEA','COPPER','GANGES','YUKON','ALSEK','SUSITNA','BALKHASH','STIKINE','SANTA CRUZ',
'FRASER','BAKER','YANGTZE','SALWEEN','COLUMBIA','ISSYK-KUL','AMAZON','COLORADO','TAKU','MACKENZIE','NASS','THJORSA','JOEKULSA A F.',
'KUSKOKWIM','RHONE','SKEENA','OB','OELFUSA','MEKONG','DANUBE','NELSON RIVER','PO','KAMCHATKA','RHINE','GLOMA','HUANG HE','INDIGIRKA',
'LULE','RAPEL','SANTA','SKAGIT','KUBAN','TITICACA','NUSHAGAK','BIOBIO','IRRAWADDY','NEGRO','MAJES','CLUTHA','DAULE-VINCES',
'KALIXAELVEN','MAGDALENA','DRAMSELV','COLVILLE']

BasinArea=[1139075,1051731,518011,1233148,64959,1024462,829632,28422,49470,423657,51147,30599,
239678,30760,1745094,258475,668561,191032,5880854,390631,17967,1752001,21211,7527,7311,
118114,97485,42944,2701040,5678,787256,793704,1099380,73066,54103,190522,42862,988062,341227,
25127,15689,11882,7961,58935,107215,29513,24108,411516,130062,18612,17118,41993,
17157,261204,17364,57544] # area of each basin in km2

basin_glacier_area = [26893.8, 24645.4, 16606.7, 15176.7, 12998., 11216., 9535.4, 5614.8, 4304., 
                      3945.4, 3467.6, 3027.8, 2495.1, 2372.3, 2317.4, 2295.9, 1878.4, 1677.3,
                      1634.1, 1601.2, 1583.6, 1519.2, 1337.3, 1251.8, 1098.6, 1032.8, 904.2, 742.3,
                      739.5, 683.4, 485.7, 408.4, 374.7, 347.3, 312.7, 285.0, 269.4, 267.9, 248.4,
                      247.2, 238.1, 198.9, 159.5, 146., 134.5, 86.4, 76.2, 71.2, 64.1, 57.3, 46.5, 
                      40.6, 37.9, 33.3, 32.1, 31.9]

yrs = np.linspace(1900, 2101, num=2412) # time interval over which we have data

Now we will load data into a dictionary for one specific climate scenario, indexed by model name.  You can choose to analyse a different climate scenario or a different SPEI integration timescale by changing the arguments `itime` and `scen` below.

In [ ]:
itime = integration_times[0] # select timescale of integration. [0] is 3 months, used in manuscript.
scen = scenarios[0] # choose whether to load RCP 4.5 or RCP 8.5

SPEI_by_model = {m: {} for m in modelnames} # create dictionary indexed by model name
for m in modelnames:
    norunoff_f_m = fpath_NP2+'NRunoff_{}_{}_{}_Conduct.txt'.format(itime, m, scen)
    wrunoff_f_m = fpath_NP2+'WRunoff_{}_{}_{}_Conduct.txt'.format(itime, m, scen)
    SPEI_by_model[m]['NRunoff'] = np.loadtxt(norunoff_f_m)
    SPEI_by_model[m]['WRunoff'] = np.loadtxt(wrunoff_f_m)
    SPEI_by_model[m]['diff'] = SPEI_by_model[m]['WRunoff'] - SPEI_by_model[m]['NRunoff']

## Re-structure dictionary and create pandas DataFrames aggregated by basin
SPEI_by_basin_raw = gSPEI.sort_models_to_basins(SPEI_by_model)
SPEI_by_basin = {b: {} for b in basin_names}
for b in basin_names:
    for c in cases:
        SPEI_by_basin[b][c] = SPEI_by_basin_raw[b][c].fillna(-3) # fill negative excursions that were stored as NaNs

## Plot multi-GCM ensemble series for a given basin

First, we can plot time series of SPEI with and without glacial runoff included.  Examples of this kind of figure appear in Ultee, Coats & Mackay Figure 1.

We will use a helper function from gSPEI.py to compare running ensemble mean SPEI with no glacial runoff (orange) versus with glacial runoff (blue).  The function can also show SPEI time series for a single basin, separated by the global climate model that produced each series.  Choose the basin by its name from the `basin_names` list, above. 

By default, we plot a running mean so that long-term trends can be more easily seen.  If you prefer, you can adjust `window` to show time series with less smoothing (shorter rolling-mean windows).  You can also turn on or off the option to see single-GCM series along with the ensembles.

In [ ]:
example_b = 'COPPER' # name of basin to examine, selected from basin_names above
win = 30 # window in years over which to calculate the running mean
plot_single_models = True # choose whether to see individual GCMs plotted over ensembles

In [ ]:
## Compute multi-GCM ensemble means and quartiles
r_w = gSPEI.basin_ensemble_mean(SPEI_by_basin, example_b, 'WRunoff').rolling(window=12*win).mean()
r_n = gSPEI.basin_ensemble_mean(SPEI_by_basin, example_b, 'NRunoff').rolling(window=12*win).mean()
rm = SPEI_by_basin[example_b]['WRunoff'].rolling(window=12*30, axis=0).mean()
rm_q1 = rm.quantile(q=0.25, axis=1)
rm_q3 = rm.quantile(q=0.75, axis=1)
rm_n = SPEI_by_basin[example_b]['NRunoff'].rolling(window=12*30, axis=0).mean()
rm_q1_n = rm_n.quantile(q=0.25, axis=1)
rm_q3_n = rm_n.quantile(q=0.75, axis=1)

single_models_w = [SPEI_by_basin[example_b]['WRunoff'][m].rolling(window=12*win).mean() for m in modelnames]
single_models_n = [SPEI_by_basin[example_b]['NRunoff'][m].rolling(window=12*win).mean() for m in modelnames]

In [ ]:
colors_w = cm.get_cmap('Blues')(np.linspace(0.2, 1, num=len(modelnames)))
colors_n = cm.get_cmap('Wistia')(np.linspace(0.2, 1, num=len(modelnames)))
fig, ax = plt.subplots()
ax.plot(yrs, r_w, 'k', linewidth=3.0)
ax.plot(yrs, rm_q1, 'k')
ax.plot(yrs, rm_q3, 'k')
ax.plot(yrs, r_n, 'k', linewidth=3.0, ls=':')
ax.plot(yrs, rm_q1_n, 'k', ls=':')
ax.plot(yrs, rm_q3_n, 'k', ls=':')
if plot_single_models:
    for i in range(len(modelnames)):
        ax.plot(yrs, single_models_w[i], color=colors_w[i], alpha=0.5)
        ax.plot(yrs, single_models_n[i], color=colors_n[i], alpha=0.5)
ax.fill_between(yrs, rm_q1, rm_q3, color='DarkBlue', alpha=0.2)
ax.fill_between(yrs, rm_q1_n, rm_q3_n, color='DarkOrange', alpha=0.2)
ax.tick_params(axis='both', labelsize=12)
ax.set_xticks([2000, 2050, 2100])
ax.set_xlim(1980, 2100)
ax.set_xlabel('Years', fontsize=14)
ax.set_ylabel('Rolling mean SPEI', fontsize=14)
plt.tight_layout()

## Compare multi-GCM ensemble series across basins

Figure 1 of Ultee, Coats & Mackay shows four example basins with different projected effects.  We reproduce that figure below.

In [ ]:
example_basins = ('COPPER', 'TARIM', 'RHONE', 'MAJES')

color_fam = cm.get_cmap('tab20b')
color_with = color_fam(0)
color_no = color_fam(10)
fig1, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex=True, 
                                              tight_layout=True, figsize=(9,6))
for example_b, ax in zip(example_basins, (ax1,ax2,ax3,ax4)):
    r_w = gSPEI.basin_ensemble_mean(SPEI_by_basin, example_b, 'WRunoff').rolling(window=12*30).mean()
    r_n = gSPEI.basin_ensemble_mean(SPEI_by_basin, example_b, 'NRunoff').rolling(window=12*30).mean()
    rm = SPEI_by_basin[example_b]['WRunoff'].rolling(window=12*30, axis=0).mean()
    rm_q1 = rm.quantile(q=0.25, axis=1, interpolation='lower')
    rm_q3 = rm.quantile(q=0.75, axis=1, interpolation='higher')
    rm_n = SPEI_by_basin[example_b]['NRunoff'].rolling(window=12*30, axis=0).mean()
    rm_q1_n = rm_n.quantile(q=0.25, axis=1, interpolation='lower')
    rm_q3_n = rm_n.quantile(q=0.75, axis=1, interpolation='higher')
    
    ax.plot(yrs, r_w, 'k', linewidth=3.0)
    ax.plot(yrs, rm_q1, 'k')
    ax.plot(yrs, rm_q3, 'k')
    ax.plot(yrs, r_n, 'k', linewidth=3.0, ls=':')
    ax.plot(yrs, rm_q1_n, 'k', ls=':')
    ax.plot(yrs, rm_q3_n, 'k', ls=':')
    # for i in range(len(modelnames)):
    #     ax.plot(yrs, single_models_w[i], color=colors_w[i], alpha=0.5)
    #     ax.plot(yrs, single_models_n[i], color=colors_n[i], alpha=0.5)
    ax.fill_between(yrs, rm_q1, rm_q3, color=color_with, alpha=0.4)
    ax.fill_between(yrs, rm_q1_n, rm_q3_n, color=color_no, alpha=0.4)
    ax.tick_params(axis='both', labelsize=12)
    ax.set_xticks([2000, 2050, 2100])
    ax.set_xlim(1980, 2100)
    extra = Rectangle((0,0), 0.1, 0.1, fc='w', fill=False, 
                      edgecolor='none', linewidth=0) # invisible marker for a legend entry
    leg = ax.legend([extra], [example_b], loc='best', 
                    handlelength=0, handletextpad=0, fancybox=True,
                    prop={'size':13})
for ax in (ax3, ax4):
    ax.set_xlabel('Year', fontsize=14)
for ax in (ax1, ax3):
    ax.set_ylabel('Rolling mean SPEI', fontsize=14)
fig1.align_ylabels()

## Multi-basin summary: change in SPEI at end of century

The functions above show timeseries and running statistics for individual basins.  But what can be said in general?  When we account for glacial runoff in our SPEI calculation, how does the mean SPEI change?  Does SPEI become more or less variable?

We calculate the mean SPEI with and without glacial runoff, for each of our 8 climate models, in each of our 56 basins, over a 30-year period.  We do the same for variance.  Then, we plot markers with whiskers to visualize the inter-model range in each.  

Figure 2 of Ultee, Coats & Mackay examines these summary results for the 30-year period at the end of the 21st century, but you can explore other periods by adjusting `timepd` below.  For simplicity, we show a single scenario here.  If you'd like to examine a different climate scenario, you can return to the top of the notebook and change the climate scenario setting `scen`.  The code to reproduce Figure 2 directly is also included in this repo, as `mean_var_shift-perscenario.py`.

In [ ]:
## Calculate changes due to glacial effect at end of century, using gSPEI functions
timepd = (2070, 2100)

## Calculate changes due to glacial effect at end of century, using ensemble approach
meandiff, quantile_spread = gSPEI.ensemble_glacial_meandiff(SPEI_by_basin, years=timepd)
vardiff, var_spread = gSPEI.ensemble_glacial_vardiff(SPEI_by_basin, years=timepd)

## Calculate full multi-GCM range, for comparison
_, mean_spread_full = gSPEI.glacial_meandiff(SPEI_by_model, years=timepd)
_, var_spread_full = gSPEI.glacial_vardiff(SPEI_by_model, years=timepd)

fig1, ax1 = plt.subplots(figsize=(5,4))
ax1.axhline(y=0, ls=':', color='Grey', alpha=0.5)
ax1.axvline(x=0, ls=':', color='Grey', alpha=0.5)
ax1.errorbar(x=meandiff, y=vardiff, xerr=quantile_spread, yerr=var_spread, 
             ls='', marker='d', elinewidth=2.0, color='k')
ax1.errorbar(x=meandiff, y=vardiff, xerr=mean_spread_full, yerr=var_spread_full, 
             ls='', marker='d', elinewidth=1.0, color='k', alpha=0.5) #extend whiskers to full range
ax1.set_xlabel('Difference in mean SPEI', fontsize=16)
ax1.set_ylabel('Difference in SPEI variance', fontsize=16)
ax1.set(ylim=(-1, 2), yticks=(-1, 0, 1, 2),
        xlim=(-0.1, 2), xticks=(0,1,2))
ax1.tick_params(axis='both', labelsize=12)
plt.tight_layout()
plt.show()

## Identifying droughts

Given the uneven changes in variance per basin, as shown above, we consider drought buffering in more granular detail.  We identify droughts as negative excursions in SPEI reaching at least -1.  Below, we use a helper function from gSPEI to mark where droughts occur, count how many occur over a time period, and find their average severity.

In [ ]:
example_basin='TARIM'
example_model='CCSM4' #show time series from a single GCM for clarity
example_period = (1980,2010)
example_series = SPEI_by_basin[example_basin]['NRunoff'][example_model]

droughts = gSPEI.find_droughts(example_series, threshold=-1, 
                               period=example_period)
drought_mask = np.full(np.shape(yrs), fill_value=False)
for k in droughts.keys():
    for n in range(k-len(droughts[k]), k+1):
        drought_mask[n] = True

In [ ]:
## Plotting
example_color=cm.get_cmap('tab20b')(10)
fig, ax = plt.subplots(figsize=(10,4), tight_layout=True)
ax.axhline(y=0, color='k', alpha=0.8)
ax.axhline(y=-1, ls='-.', color='k')
ax.plot(yrs, example_series, color=example_color)
ax.fill_between(yrs, y1=0, y2=example_series, where=drought_mask, 
                alpha=0.5, color=example_color)
ax.annotate('{} droughts found'.format(len(droughts)), xy=(2000,-3),
            fontweight='bold')
ax.set(xlim=example_period)
ax.tick_params(axis='both', labelsize=12)
ax.set_xlabel('Year', fontsize=14)
ax.set_ylabel(r'SPEI$_N$', fontsize=14)
plt.show()

We can also compare the droughts found for the same example basin and model, with versus without glacial runoff.

In [ ]:
example_series_n = SPEI_by_basin[example_basin]['NRunoff'][example_model]
example_series_w = SPEI_by_basin[example_basin]['WRunoff'][example_model]

example_color_n=cm.get_cmap('tab20b')(10)
example_color_w=cm.get_cmap('tab20b')(0)

droughts_n = gSPEI.find_droughts(example_series_n, threshold=-1, 
                               period=example_period)
drought_mask_n = np.full(np.shape(yrs), fill_value=False)
for k in droughts_n.keys():
    for n in range(k-len(droughts_n[k]), k+1):
        drought_mask_n[n] = True
droughts_w = gSPEI.find_droughts(example_series_w, threshold=-1, 
                               period=example_period)
drought_mask_w = np.full(np.shape(yrs), fill_value=False)
for k in droughts_w.keys():
    for n in range(k-len(droughts_w[k]), k+1):
        drought_mask_w[n] = True
        
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12,6), tight_layout=True)
ax1.axhline(y=0, color='k', alpha=0.8)
ax1.axhline(y=-1, ls='-.', color='k')
ax1.plot(yrs, example_series_n, color=example_color_n)
ax1.fill_between(yrs, y1=0, y2=example_series_n, where=drought_mask_n, 
                alpha=0.5, color=example_color_n)
ax1.annotate('{} droughts found'.format(len(droughts_n)), xy=(2000,-3),
            fontweight='bold')
ax1.set(xlim=example_period, yticks=(-2, 0, 2))
ax1.tick_params(axis='both', labelsize=12)
ax1.set_xlabel('Year', fontsize=14)
ax1.set_ylabel(r'SPEI$_N$', fontsize=14)

ax2.axhline(y=0, color='k', alpha=0.8)
ax2.axhline(y=-1, ls='-.', color='k')
ax2.plot(yrs, example_series_w, color=example_color_w)
ax2.fill_between(yrs, y1=0, y2=example_series_w, where=drought_mask_w, 
                alpha=0.5, color=example_color_w)
ax2.annotate('{} droughts found'.format(len(droughts_w)), xy=(2000,-3),
            fontweight='bold')
ax2.set(xlim=example_period, yticks=(-2,0,2))
ax2.tick_params(axis='both', labelsize=12)
ax2.set_xlabel('Year', fontsize=14)
ax2.set_ylabel(r'SPEI$_W$', fontsize=14)
plt.show()


## Glacial drought buffering over time

Now we summarize the number and severity of droughts counted in all basins, for all GCMs.  We contrast the SPEI$_N$ series versus SPEI$_W$ series to isolate glacial drought buffering on each metric.

We use another helper function to identify the droughts in each series and summarize them all, storing each in a dictionary per time period such that we can see change over time.

In [ ]:
basin_stats_bymodel_hist = {m: {b: gSPEI.basin_summary_stats(SPEI_by_basin, basin_name=b, modelnames=[m], period=(1980,2010)) for b in basin_names} 
                    for m in modelnames}
basin_stats_bymodel_midC = {m: {b: gSPEI.basin_summary_stats(SPEI_by_basin, basin_name=b, modelnames=[m], period=(2030,2060)) for b in basin_names} 
                    for m in modelnames}
basin_stats_bymodel_endC = {m: {b: gSPEI.basin_summary_stats(SPEI_by_basin, basin_name=b, modelnames=[m], period=(2070,2100)) for b in basin_names} 
                    for m in modelnames}

The plot is complicated to generate.  Hang in there!

In [ ]:
## Composite of all stats over time
color_fam = cm.get_cmap('tab20b')
inc_color=color_fam(5)
dec_color=color_fam(17)

fig3, ((ax1,ax2, ax3), 
       (ax7,ax8,ax9)) = plt.subplots(2,3, sharex=True, sharey='row', figsize=(12,8))
for b, a, ag in zip(basin_names, BasinArea, basin_glacier_area):
    pg = ag/a # percent glaciated
    number_b = []
    duration_b = []
    severity_b = []
    number_midC = []
    number_endC = []
    duration_midC = []
    duration_endC = []
    severity_midC = []
    severity_endC = []

    for m in modelnames:
        number_b.append(basin_stats_bymodel_hist[m][b][0][1]-basin_stats_bymodel_hist[m][b][0][0])
        number_midC.append(basin_stats_bymodel_midC[m][b][0][1]-basin_stats_bymodel_midC[m][b][0][0])
        number_endC.append(basin_stats_bymodel_endC[m][b][0][1]-basin_stats_bymodel_endC[m][b][0][0])
        severity_b.append(-1*(basin_stats_bymodel_hist[m][b][2][1]-basin_stats_bymodel_hist[m][b][2][0]))
        severity_midC.append(-1*(basin_stats_bymodel_midC[m][b][2][1]-basin_stats_bymodel_midC[m][b][2][0]))
        severity_endC.append(-1*(basin_stats_bymodel_endC[m][b][2][1]-basin_stats_bymodel_endC[m][b][2][0]))
   
    ## Color code changes over time
    midC_v_hist_n = np.nanmean(number_midC)-np.nanmean(number_b)
    if midC_v_hist_n >0.1: # buffering on number increasing
        midC_color_n=inc_color
        midC_marker_n='^'
    elif midC_v_hist_n<-0.1:
        midC_color_n=dec_color
        midC_marker_n='v'
    else:
        midC_color_n='k'
        midC_marker_n='o'
    endC_v_hist_n = np.nanmean(number_endC)-np.nanmean(number_b)
    if endC_v_hist_n >0.1:
        endC_color_n=inc_color
        endC_marker_n='^'
    elif endC_v_hist_n<-0.1:
        endC_color_n=dec_color
        endC_marker_n='v'
    else:
        endC_color_n='k'
        endC_marker_n='o'
        
    midC_v_hist_d = np.nanmean(duration_midC)-np.nanmean(duration_b)
    if midC_v_hist_d >0.1: # buffering on duration increasing
        midC_color_d=inc_color
        midC_marker_d='^'
    elif midC_v_hist_d<-0.1:
        midC_color_d=dec_color
        midC_marker_d='v'
    else:
        midC_color_d='k'
        midC_marker_d='o'
    endC_v_hist_d = np.nanmean(duration_endC)-np.nanmean(duration_b)
    if endC_v_hist_d >0.1:
        endC_color_d=inc_color
        endC_marker_d='^'
    elif endC_v_hist_d<-0.1:
        endC_color_d=dec_color
        endC_marker_d='v'
    else:
        endC_color_d='k'
        endC_marker_d='o'
        
    midC_v_hist_s = np.nanmean(severity_midC)-np.nanmean(severity_b)
    if midC_v_hist_s >0.1: # buffering on duration increasing
        midC_color_s=inc_color
        midC_marker_s='^'
    elif midC_v_hist_s<-0.1:
        midC_color_s=dec_color
        midC_marker_s='v'
    else:
        midC_color_s='k'
        midC_marker_s='o'
    endC_v_hist_s = np.nanmean(severity_endC)-np.nanmean(severity_b)
    if endC_v_hist_s >0.1:
        endC_color_s=inc_color
        endC_marker_s='^'
    elif endC_v_hist_s<-0.1:
        endC_color_s=dec_color
        endC_marker_s='v'
    else:
        endC_color_s='k'
        endC_marker_s='o'
    ## First column: historical
    ax1.errorbar(pg, np.nanmean(number_b), 
                 yerr=((np.nanmean(number_b)-np.nanmin(number_b), np.nanmax(number_b)-np.nanmean(number_b)),), 
                 color='k', marker='o', lw=1.0)
    ax7.errorbar(pg, np.nanmean(severity_b), 
                 yerr=(((np.nanmean(severity_b)-np.nanmin(severity_b), np.nanmax(severity_b)-np.nanmean(severity_b)),)), 
                 color='k', marker='o', lw=1.0)
    ## Second column: mid-c
    ax2.errorbar(pg, np.nanmean(number_midC), 
                 yerr=(((np.nanmean(number_midC)-np.nanmin(number_midC), np.nanmax(number_midC)-np.nanmean(number_midC)),)), 
                 color=midC_color_n, marker=midC_marker_n, lw=1.0)
    ax8.errorbar(pg, np.nanmean(severity_midC), 
                 yerr=(((np.nanmean(severity_midC)-np.nanmin(severity_midC), np.nanmax(severity_midC)-np.nanmean(severity_midC)),)), 
                 color=midC_color_s, marker=midC_marker_s, lw=1.0)
    ## Third column: end of century
    ax3.errorbar(pg, np.nanmean(number_endC), 
                 yerr=(((np.nanmean(number_endC)-np.nanmin(number_endC), np.nanmax(number_endC)-np.nanmean(number_endC)),)), 
                 color=endC_color_n, marker=endC_marker_n, lw=1.0)
    ax9.errorbar(pg, np.nanmean(severity_endC), 
                 yerr=(((np.nanmean(severity_endC)-np.nanmin(severity_endC), np.nanmax(severity_endC)-np.nanmean(severity_endC)),)), 
                 color=endC_color_s, marker=endC_marker_s, lw=1.0)

ax1.set(ylabel=r'$\Delta$ Number', title='Historical (1980-2010)', xscale='log')
ax2.set(title='Mid-21st Cent. (2030-2060)')
ax3.set(title='End 21st Cent. (2070-2100)')
ax7.set(ylabel=r'$\Delta$ Severity', 
        xscale='log',xlabel='Glacier area fraction', xlim=(1E-4, 0.23))
ax8.set(xlabel='Glacier area fraction', xlim=(1E-4, 0.23))
ax9.set(xlabel='Glacier area fraction', xlim=(1E-4, 0.23))
for ax in (ax1,ax2,ax3,
           ax7,ax8,ax9):
    ax.axhline(0, ls=':', lw=1.0, color='k')
    

Magenta downward triangles indicate decreasing buffering.  Green upward triangles indicate increasing buffering.

## K-means clustering

For k-means clustering, which uses different packages, see `kmeans.py` in the repository.
[EHU TO UPDATE LATER]